In [1]:
from collections import Counter

In [2]:
def compute_extended_gcd(a, m):
    if a == 0:
        return m, 1, 0
    gcd, x, y = compute_extended_gcd(m % a, a)
    coefficient = x - (m // a) * y
    return gcd, y, coefficient

def find_modular_inverse(a, m):
    gcd, inverse, coefficient = compute_extended_gcd(a, m)
    result = [gcd, inverse, coefficient]
    return result


In [4]:
def solve_congruence(a, b, m):
    inverse_result = find_modular_inverse(a, m)
    gcd = inverse_result[0]

    if gcd == 1:
        coefficient = inverse_result[2]
        return (b * coefficient) % m
    elif gcd > 1 and b % gcd == 0:
        reduced_inverse = find_modular_inverse(a // gcd, m // gcd)
        reduced_gcd = reduced_inverse[0]
        coefficient = reduced_inverse[2]
        base_solution = (b // gcd * coefficient) % (m // gcd)
        return [base_solution + (m // gcd) * i for i in range(gcd)]


In [5]:
def most_frequent_bigrams_with_relative_frequency(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()
    filtered_text = ''.join(char.lower() for char in content if char.isalnum() or char.isspace())
    bigrams = [filtered_text[i:i+2] for i in range(len(filtered_text) - 1)]
    bigram_frequency = Counter(bigrams)
    total_count = sum(bigram_frequency.values())
    top_5_bigrams = bigram_frequency.most_common(5)

    for bigram, count in top_5_bigrams:
        frequency_percentage = count / total_count
        print(f"'{bigram}': {count} (frequency: {frequency_percentage:.2%})")

most_frequent_bigrams_with_relative_frequency('01.txt')


'рн': 63 (frequency: 1.28%)
'ыч': 44 (frequency: 0.89%)
'нк': 43 (frequency: 0.87%)
'цз': 37 (frequency: 0.75%)
'тч': 33 (frequency: 0.67%)


In [6]:
cipher_bigrams = ['рн', 'ыч', 'нк', 'цз', 'тч']
russian_bigrams = ['ст', 'но', 'то', 'на', 'ен']
alphabet = 'абвгдежзийклмнопрстуфхцчшщьыэюя'
letter_to_index = {char: idx for idx, char in enumerate(alphabet)}
modulus = 961

bigram_pairs = [
    (russian_bigram, cipher_bigram)
    for russian_bigram in russian_bigrams
    for cipher_bigram in cipher_bigrams
]
print("Bigram pairs:", bigram_pairs)

numeric_bigram_pairs = [
    (
        31 * letter_to_index[pair[0][0]] + letter_to_index[pair[0][1]],
        31 * letter_to_index[pair[1][0]] + letter_to_index[pair[1][1]]
    )
    for pair in bigram_pairs
]
print("Numeric bigram pairs:", numeric_bigram_pairs)

potential_keys = []
for i in range(len(numeric_bigram_pairs)):
    X1, Y1 = numeric_bigram_pairs[i]

    for j in range(i + 1, len(numeric_bigram_pairs)):
        X2, Y2 = numeric_bigram_pairs[j]
        delta_X = X1 - X2
        delta_Y = Y1 - Y2

        if delta_X == 0:
            continue

        try:
            inverse_result = find_modular_inverse(delta_X, modulus)
            inv_delta_X = inverse_result[2]
            a = (delta_Y * inv_delta_X) % modulus
            b = (Y1 - a * X1) % modulus
            key = solve_congruence(a, b, modulus)
            if key is not None:
                potential_keys.append((a, b))
        except ValueError:
            continue

print("\nPotential keys:")
for key in potential_keys:
    print(key)


Bigram pairs: [('ст', 'рн'), ('ст', 'ыч'), ('ст', 'нк'), ('ст', 'цз'), ('ст', 'тч'), ('но', 'рн'), ('но', 'ыч'), ('но', 'нк'), ('но', 'цз'), ('но', 'тч'), ('то', 'рн'), ('то', 'ыч'), ('то', 'нк'), ('то', 'цз'), ('то', 'тч'), ('на', 'рн'), ('на', 'ыч'), ('на', 'нк'), ('на', 'цз'), ('на', 'тч'), ('ен', 'рн'), ('ен', 'ыч'), ('ен', 'нк'), ('ен', 'цз'), ('ен', 'тч')]
Numeric bigram pairs: [(545, 509), (545, 860), (545, 413), (545, 689), (545, 581), (417, 509), (417, 860), (417, 413), (417, 689), (417, 581), (572, 509), (572, 860), (572, 413), (572, 689), (572, 581), (403, 509), (403, 860), (403, 413), (403, 689), (403, 581), (168, 509), (168, 860), (168, 413), (168, 689), (168, 581)]

Potential keys:
(230, 89)
(241, 821)
(389, 885)
(540, 275)
(948, 867)
(751, 600)
(634, 939)
(638, 681)
(275, 550)
(894, 506)
(486, 875)
(771, 271)
(463, 917)
(13, 151)
(336, 940)
(711, 297)
(731, 319)
(11, 631)
(159, 695)
(13, 502)
(764, 593)
(647, 932)
(686, 819)
(619, 816)
(211, 224)
(498, 452)
(511, 94)
(83

In [7]:
char_to_index = {char: idx for idx, char in enumerate(alphabet)}
index_to_char = {idx: char for idx, char in enumerate(alphabet)}

with open('01.txt', 'r', encoding='utf-8') as file:
    cipher_text = file.read().replace('\n', '').replace('\r', '')
cipher_indexes = [char_to_index[char] for char in cipher_text]

def affine_decrypt(cipher_indexes, a, b, mod):
    mod_squared = mod ** 2
    decrypted_output = []

    try:
        inverse_result = find_modular_inverse(a, mod_squared)
        a_inverse = inverse_result[2]
    except ValueError:
        print(f"Inverse not found. Skipping key.")
        return None

    for i in range(0, len(cipher_indexes), 2):
        y = cipher_indexes[i]
        next_y = cipher_indexes[i + 1] if i < len(cipher_indexes) - 1 else 0
        combined_index = y * 31 + next_y
        x = (a_inverse * (combined_index - b)) % mod_squared
        x1, x2 = (x // 31), (x % 31)

        decrypted_output.append(index_to_char[x1 % mod])
        decrypted_output.append(index_to_char[x2 % mod])

    return ''.join(decrypted_output)

decrypted_texts = {}
for a, b in potential_keys:
    decryption_result = affine_decrypt(cipher_indexes, a, b, len(alphabet))
    if decryption_result:
        decrypted_texts[(a, b)] = decryption_result


In [10]:
rare_bigrams = ["щт", "ьо", "ыж", "юв", "яы", "аы", "бй", "гй", "дй", "еы", "шщ", "шя", "щб", "щд", "щж", "ьы", "ыа", "ыь", "ыы", "ыэ"]

def split_text_into_bigrams(text):
    return [text[i:i+2] for i in range(0, len(text), 2)]

def count_rare_bigrams_in_text(text, rare_bigrams_list):
    bigrams = split_text_into_bigrams(text)
    return sum(1 for bigram in bigrams if bigram in rare_bigrams_list)

bigram_frequency_count = {}
for key, decrypted_text in decrypted_texts.items():
    rare_bigram_count = count_rare_bigrams_in_text(decrypted_text, rare_bigrams)
    bigram_frequency_count[key] = rare_bigram_count


sorted_bigrams_by_count = sorted(bigram_frequency_count.items(), key=lambda x: x[1])

print("Ключ з найменшою кількістю рідкісних біграм:")
for i, (key, count) in enumerate(sorted_bigrams_by_count[:1]):
    print(f"{i + 1}. Ключ {key} - Кількість рідкісних біграм: {count}")


Ключ з найменшою кількістю рідкісних біграм:
1. Ключ (13, 151) - Кількість рідкісних біграм: 3


In [9]:
end_key = (13, 151)
print(decrypted_texts[end_key])

многограннуюличностьдостоевскогоможнорассматриватьсчетырехсторонкакписателякакневротикакакмыслителяэтикаикакгрешникакакжеразобратьсявэтойневольносмущающейнассложностинаименееспоренонкакписательместоеговодномрядусшекспиромбратьякарамазовывеличайшийроманизвсехкогдалибонаписанныхалегендаовеликоминквизитореодноизвысочайшихдостижениймировойлитературыпереоценитькотороеневозможноксожалениюпередпроблемойписательскоготворчествапсихоанализдолженсложитьоружиедостоевскийскореевсегоуязвимкакморалистпредставляяегочеловекомвысоконравственнымнатомоснованиичтотолькототдостигаетвысшегонравственногосовершенствактопрошелчерезглубочайшиебездныгреховностимыигнорируемодносоображениеведьнравственнымявляетсячеловекреагирующийуженавнутреннеиспытываемоеискушениеприэтомемунеподдаваяськтожепопеременнотогрешиттораскаиваясьставитсебевысокиенравственныецелитоголегкоупрекнутьвтомчтоонслишкомудобнодлясебястроитсвоюжизньоннеисполняетосновногопринципанравственностинеобходимостиотречениявтовремякакнравственныйобразжизнивп